# 5.3 RNN 구현            

### 1. 목표

- 우리가 지금부터 구현할 것은 결국 '가로 방향으로 성장한 신경망'!       

![rnn1](./PostingPic/rnn_1.png)       


- 길이가 T인 시계열 데이터를 받아서, 각 시각의 은닉 상태를 T개 출력한다.
- 상하 방향의 입력과, 출력을 하나로 묶으면 옆으로 늘어선 일련의 계층을 하나의 계층으로 간주할 수 있다.  
- 즉, 묶어주는 $Xs$ 로 묶여진 $Hs$를 출력하는 하나의 계층

### 2. 구현           

- 한 단위의 RNN 클래스를 구현해보자.    
- 여기서, 우리는 데이터를 미니배치로 처리한다.      

![rnn2](./PostingPic/rnn_2.png)

ht(한 계층의 출력) = tahn(ht-1 * Wh + Xt * Wx + b)      

> - h(t-1) * Wh : 이전 단계의 h값과, Wh(h 출력을 내기 위한 가중치 세트)      
> - Xt * Wx : 현재 단계의 인풋 Xt와 Wx(h값을 내기 위한 가중치 세트)        
> - b : 편향    

##### 순전파

In [ ]:
class RNN:
    #파라미터로 Wx, Wh의 두 값이 필요
    def __init__(self, Wx, Wh, b):
        self.params[Wx, Wh, b]
        
        #가중치는 요렇게
        #동일 크기의 제로 행렬을 만들어준다.
        self.grads[np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        
        #다음 계층으로 넘어갈 때 기억해야 할 값인데
        #현재는 아무것도 알 수 없으므로 None
        self.cache = None
        
    #순방향 전파
    #x(각 시각별 인풋)
    #h_prev(이전 계층의 출력)
    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        
        # 이 부분이 바로 그 그림입니다
        t = np.matmul(h_prev, Wh) + np.matmul(x, Wx) + b
        h_next = np.tanh(t)
        
        self.cache = (x, h_prev, h_next)
        
        #이번 층에서 나온 결과값인 h_next를 리턴
        return h_next

![순전파그래프](./PostingPic/순전파그래프.png)

##### 역전파   

In [ ]:
# 순전파일때는 앞서있었던 그 값
# 역전파일때는 거꾸로 온겁니다!
def backward(self, dh_next):
    Wx, Wh, b = self.params
    x, h_prev, h_next = self.cache
    
    dt = dh_next * (1-h_next ** 2)
    
    #편향의 d
    db = np.sum(dt, axis=0)
    
    #Wh의 d
    dWh = np.matmul(h_prev.T, dt)
    
    #h_prev의 d
    dh_prev = np.matmul(dt, Wh.T)
    
    #Wx의 d
    dWx = np.matmul(x.T, dt)
    
    #x의 d
    dx = np.matmul(dt, Wx.T)
    
    #전달해야 할 기울기 Wx, Wh, b
    self.grad[0][] = dWx
    self.grad[1][] = dWh
    self.grad[2][] = db
    
    return dx, dh_prev

## 5.3.2 TimeRnn계층의 구현                 

- Time Rnn계층 == Rnn * T개         
- Rnn 계층의 은닉 상태인 h를 인스턴스 변수로 사용    

![TimeRnn](./PostingPic/TimeRnn.png)            


- 이 책에서는 '은닉 상태를 인계받을지'를 stateful 이라는 인수로 조정한다.      
- Stateful Stateless

![예제](./PostingPic/State.png)             


##### 순전파

In [ ]:
class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None
        
        #이번 계층의 결과물, h의 기울기
        self.h, self.dh = None, None
        
        #연결 상태는 어떻게 할거니?
        self.stateful = stateful
        
        
    def set_state(self, h):
        self.h = h
        
    def reset_state(self):
        self.h = None

In [ ]:
#이 계층은 여러 개의 x, h를 묶은 것이라는 것을 기억하자!
    def forward(self, xs):
        Wx, Wh, b = self.params
        #N = 미니배치, T=rnn개수, D=입력 벡터의 차원 수 
        N, T, D = xs.shape
        D, H = Wx.shape
        
        #레이어를 리스트로 담아둘거야.
        self.layers = []
        
        #결과물을 담아둘 수 있는 행렬을 만듦
        hs = np.empty((N, T, H), dtype='f')
        
        #스테이트가 true이고 h가 설정되어 있지 않으면
        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H), dtype='f')
            
        for t in range(T):
            # *는 리스트의 원소를 추출하여 메서드의 인수로 전달한다.
            # self.params = [Wx, Wh, b]
            layer = RNN(*self.params)
            self.h = layer.forward(xs[:,t,:], self.h)
            
            #  N, T, H
            hs[:, t, :] = self.h
            self.layers.append(layer)
            
        return hs

![이렇게](./PostingPic/rnn_3.png)

##### 역전파           

- 우리는 Truncated BPTT(잘린 BPTT)를 수행하므로, 이전 시각 역전파는 기억하지 않는다. (한 단위에서 끝)   

![이렇게](./PostingPic/TimeRnn_bptt.png)

In [ ]:
def backpropagation(self, dhs):
    Wx, Wh, b = self.params
    
    # 기울기 셰입
    N, T, H = dhs.shpae
    D, H = Wx.shape
    
    #모든 기울기는 0으로 초기화 세팅
    dxs = np.empty((N,T,D), dtype='f')
    dh = 0
    grads =[0,0,0]
    
    for t in reversed(range(T)):
        layer = self.layers[t]
        
        dx, dh = layer.backward(dhs[:, t, :] + dh)
        
        #전체 인풋 xs에 대한 기울기
        dxs = dx
        
        for i, grad in enumerate(layer.grads):
            grads[i] +=  grad
            
    for i, grad in enumerate(layer.grads):
        self.grads[i][] = grad
    
    self.dh = dh
    
    return dxs

- 단, 순전파 시에 갈라졌던 부분들을 합산해서 적용해야 함

### 5.4 시계열 데이터 처리 계층 구현          

- RNNLM (RNN + LM)       

![모델](./PostingPic/언어모델.png)

- 임베딩 층 : 단어 ID를 단어 벡터(분산표현)으로 변환         
- RNN : 우리가 지금까지 한 것      
- Affine, Softmax : 마지막 처리

![여기](./PostingPic/5-26.png)

- Say를 보자구   
- hello, goodbye 둘 다 확률이 높게 나옴(둘 다 정답이 될 만한 단어들)        
- You say goodbye and I say hello  의 순서를 __기억하고__ 있다는 것         

##### 2. Time embedding 구현, Time Affine 구현              

[전체 코드는 여기에](https://github.com/WegraLee/deep-learning-from-scratch-2/blob/master/common/time_layers.py)

In [ ]:
class TimeEmbedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.layers = None
        self.W = W

    def forward(self, xs):
        N, T = xs.shape
        V, D = self.W.shape

        out = np.empty((N, T, D), dtype='f')
        self.layers = []

        for t in range(T):
            layer = Embedding(self.W)
            out[:, t, :] = layer.forward(xs[:, t])
            self.layers.append(layer)

        return out

    def backward(self, dout):
        N, T, D = dout.shape

        grad = 0
        for t in range(T):
            layer = self.layers[t]
            layer.backward(dout[:, t, :])
            grad += layer.grads[0]

        self.grads[0][...] = grad
        return None

In [ ]:
class TimeAffine:
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None

    def forward(self, x):
        N, T, D = x.shape
        W, b = self.params

        rx = x.reshape(N*T, -1)
        out = np.dot(rx, W) + b
        self.x = x
        return out.reshape(N, T, -1)

    def backward(self, dout):
        x = self.x
        N, T, D = x.shape
        W, b = self.params

        dout = dout.reshape(N*T, -1)
        rx = x.reshape(N*T, -1)

        db = np.sum(dout, axis=0)
        dW = np.dot(rx.T, dout)
        dx = np.dot(dout, W.T)
        dx = dx.reshape(*x.shape)

        self.grads[0][...] = dW
        self.grads[1][...] = db

        return dx

##### 3. simple RnnLm                

- 4개의 계층을 쌓은 것

In [ ]:
import sys
sys.path.append('..')
import numpy as np
from common.time_layers import *

#F: 포트란형식 Index 순서, 앞 차원부터 변경하고 뒷 쪽 차원을 변경
class SimpleRnnlm:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn

        # 가중치 초기화
        embed_W = (rn(V, D) / 100).astype('f')
        rnn_Wx = (rn(D, H) / np.sqrt(D)).astype('f')
        rnn_Wh = (rn(H, H) / np.sqrt(H)).astype('f')
        rnn_b = np.zeros(H).astype('f')
        affine_W = (rn(H, V) / np.sqrt(H)).astype('f')
        affine_b = np.zeros(V).astype('f')

        # 계층 생성
        self.layers = [
            TimeEmbedding(embed_W),
            TimeRNN(rnn_Wx, rnn_Wh, rnn_b, stateful=True),
            TimeAffine(affine_W, affine_b)
        ]
        self.loss_layer = TimeSoftmaxWithLoss()
        self.rnn_layer = self.layers[1]

        # 모든 가중치와 기울기를 리스트에 모은다.
        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, xs, ts):
        for layer in self.layers:
            xs = layer.forward(xs)
        loss = self.loss_layer.forward(xs, ts)
        return loss

    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout

    def reset_state(self):
        self.rnn_layer.reset_state()

- RNN과 Affine 계층에서 Xavier 초깃값 (6.2 가중치의 초깃값)

### 5.5.2 언어모델의 평가                 

##### 퍼플렉시티(Perplexity, 혼란도)           
- 확률의 역수         
- 각 언어 벡터에 대한 정답 확률을, 거꾸로 생각해보는 것       

![퍼플렉시티](./PostingPic/퍼플렉시티.png)

- 직관적으로 생각할 때, 분기 수로 해석할 수 있다.           
- 내가 고를 수 있는 선택지가 몇 개인가?             
- 정답 확률이 낮은 경우(0.1퍼일 때) = > 퍼플렉시티는 10. 이는 곧 선택가능한 선택지 10개를 주며 이중에 정답이 있어~ 하는 것

### 질문 모음

(진호) 
Q1. Truncate BPTT 적당한 길이란 어떻게 되는지? BPTT 어떤 기준으로 잡을 수 있을까요?
A1. https://machinelearningmastery.com/truncated-backpropagation-through-time-in-keras/ 의 2번을 보시면 어떨까요?

(병학)
Q1. RNN 계층 모두가 실제로는 '같은 계층'이라고 하는것이 X0 ~Xt 가 RNN층을 만나 h0~ht로 변하는기 때문에 같은 계층이라고 하는 것인가요? 
A1. (스레드로 알려주세요!)

(창원)
Q1.(p202) RNN 계층 모두가 실제로는 '같은 계층': 그림 5-8에 있는 것은 하나의 계층인가?
A1. (스레드로 알려주세요!)

Q2.(p202) 식 5.9에서 tanh를 사용하는 이유?
A2. The vanishing gradient problem is the main problem in RNN. Also, to keep the gradient in the linear region of the activation function, we need a function whose second derivative can sustain for a long range before going to zero. Tanh is pretty good with these properties. 라고 합니다.

Q3. (p204) 시간 크기가 커지면 역전파 시의 기울기가 불안정해지는 이유?
A3. 미분이기 때문에 점점 줄어들고, 멀리까지 값이 퍼지기 어렵기 때문인 것 같습니다.

Q4.(p208~209) Truncated BPTT의 미니배치 학습에 대해 설명해주세요
A4. (스레드로 알려주세요!)

(상미)
Q1.(p205) 신경망 하나씩 통과할 때마다, 기울기 값이 작아져 불안정해진다고 이해했습니다. 이때 '역전파 시 기울기가 불안정해진다', '역전파 일때만 끊어준다'와 같이 역전파에만 해당시키는 이유가 있을까요?
A1. (스레드로 알려주세요!)

(창호)
Q1. hidden state에는 가중치가 왜 2개 필요한가? 그것이 의미하는 게 무엇인가?
A1. (스레드로 알려주세요!)

Q2. 자연어를 반드시 시계열 데이터라고만 봐야할까?
A2. 문맥의 순서가 있으므로 시계열로 볼 수 있을 거라 생각한다.
    현재에 가치를 더 둘건지 과거의 데이터에 가치를 더 둘 건지?

(준형)
Q1. (p201) RNN 순환구조에 입력되는 데이터(x0, x1, x2....xt) 각각은 말뭉치 전체 데이터인가요 아니면 맥락인가요? 
A1.193쪽 그림 5-2가 도움이 될 것 같습니다.
   Exploration 작사가 노드를 보시면 좋을 것 같습니다!

(정은) 
Q1. 왜 Xavier 초깃값을 썼을까?     
A1. −m과 +m 사이의 균등 분포를 의미합니다. 세이비어 초기화는 여러 층의 기울기 분산 사이에 균형을 맞춰서 특정 층이 너무 주목을 받거나 다른 층이 뒤쳐지는 것을 막습니다. Xavier 초기화는 시그모이드 함수나 하이퍼볼릭 탄젠트 함수와 같은 S자 형태인 활성화 함수와 함께 사용할 경우에는 좋은 성능을 보입니다. 

(상효) 
Q1. 사람의 기억상실,운동의 소실과 같이 시간이 흐름에 따라 발생하는 퇴행성 질환의 경우 
시계열 데이터를 다루는 RNN모델의 구조의처리방식과 상관관계가 있을까요 ? 
상관관계가 있다면 반대로 RNN모델을 이용하여 퇴행성 뇌질환의 조기 진단이 가능할까요 ?

A1. https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART002468561
2. 노년기 3대 질환 중 하나인 파킨슨병은 환자의 70 % 이상이 음성 장애를 앓고 있으며 최근 음성 신호를 통한파킨슨병의 진단 방법들이 고안되고 있다. 본 논문에서는 음성 특징을 이용한 심층 잔류 순환 신경망 기반의 파킨슨병진단 방식을 제안한다. 제안하는 방식에서는 파킨슨병 진단을 위한 음성 특징을 선택하고 이를 심층 잔류 순환 신경망에 적용하여 파킨슨병 환자를 식별한다. 제안하는 심층 잔류 순환 신경망은 심층 순환 신경망에 잔류 학습 방식을 결합한 알고리즘으로 파킨슨병 진단에서 기존의 식별 알고리즘보다 더 높은 인식률을 보인다.